# PIC-SURE python API use-case: Phenome-Wide analysis on BioData Catalyst studies

This notebook is an illustration example about how to query data using the python **PIC-SURE API**. It takes as use-case a simple PheWAS analysis. This notebook is intentionally straightforward, and explanation provided are only aimed at guiding through the PheWAS analysis process. For a more step-by-step introduction to the python PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to get a user-specific security token. For more information on how to proceed, see the `get_your_token.ipynb` notebook**

# Environment set-up

### System requirements
- Python 3.6 or later
- pip package manager
- bash interpreter

### Installation of external dependencies

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

#### Installing latest python PIC-SURE API libraries from github

In [ ]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureClient
import PicSureBdcAdapter

from python_lib.utils import get_multiIndex_variablesDict, joining_variablesDict_onCol

In [ ]:
print("NB: This Jupyter Notebook has been written using PIC-SURE API following versions:\n- PicSureBdcAdapter: 1.0.0\n- PicSureClient: 1.1.0")
print("The installed PIC-SURE API libraries versions:\n- PicSureBdcAdapter: {0}\n- PicSureClient: {1}".format(PicSureBdcAdapter.__version__, PicSureClient.__version__))

In [ ]:
# Pandas DataFrame display options
pd.set_option("max.rows", 100)

# Matplotlib display parameters
plt.rcParams["figure.figsize"] = (14,8)
font = {'weight' : 'bold',
        'size'   : 12}
plt.rc('font', **font)

## Connecting to a PIC-SURE network

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"

In [ ]:
with open(token_file, "r") as f:
    my_token = f.read()

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token)
adapter = PicSureBdcAdapter.Adapter(connection)
resource = adapter.useResource(resource_id)

# PheWAS analysis
*Note: This example is not meant to be publication-ready, but rather serve as a guide or starting point to perform PheWAS.*

This PheWAS analysis focuses on the TopMed DCC Harmonized Variables. 
We leverage the harmonized variables to provide an example PheWAS focused on total cholesterol in two studies: ARIC and FHS.
The PIC-SURE API is helpful in wrangling our phenotypic data. 

In a nutshell, this PheWAS analysis follows the subsequent steps:
1. Retrieving the variable dictionary, using the PIC-SURE API dedicated methods
2. Using the PIC-SURE API to select variables and retrieve data
3. Data management
4. Statistical analysis for each study and sex
5. Visualization of results in Manhattan Plot

With this, we are tackling two different analysis considerations of a PheWAS: 
1. Using multiple variables in a PheWAS. In this example, we are looking into sex differences of total 
2. Harmonization and meta-analysis issues when using data from multiple studies or datasets

### 1. Retrieving variable dictionary from PIC-SURE
The first step to conducting the PheWAS is to retrieve information about the variables that will be used in the analysis. For this example, we will be using variables from the TOPMed Data Coordinating Center (DCC) Harmonized data set. 

The Data Harmonization effort aims to produce "a high quality, lasting resource of publicly available and thoroughly documented harmonized phenotype variables". The TOPMed DCC collaborates with Working Group members and phenotype experts on this endeavour. So far, 44 harmonized variables are accessible through PIC-SURE API (in addition to the age at which each variable value has been collected for a given subject).

Which phenotypic characteristics are included in the harmonized variables?

- Key NHLBI phenotypes
    - Blood cell counts
    - VTE
    - Atherosclerosis-related phenotypes
    - Lipids
    - Blood pressure
    
    
- Common covariates
    - Height
    - Weight
    - BMI
    - Smoking status
    - Race/ethnicity

More information about the variable harmonization process is available at https://www.nhlbiwgs.org/sites/default/files/pheno_harmonization_guidelines.pdf

Here, we retrieve the harmonized variables information by searching for `DCC Harmonized data set` in PIC-SURE.

In [ ]:
harmonized_variables = resource.dictionary().find("DCC Harmonized data set")

In [ ]:
harmonized_dic = harmonized_variables.DataFrame()

In [ ]:
# Display the variables tree hierarchy from the variables name
variablesDict = get_multiIndex_variablesDict(harmonized_dic)
variablesDict.head()

As you can see, we retrieved 80 variables from the DCC harmonized data set. The structure of `variablesDict` allows us to visualize the tree-like structure of the concept paths more easily.

### 2. Using the PIC-SURE API to select variables and retrieve data
Now that we've retrieved the variable information, we need to select our variable of interest. In this example, we are interested in exploring the relationship between the harmonized variables and blood cholesterol. Specifically, we will find the full concept path that contains "Blood mass concentration of total cholesterol".

In [ ]:
my_harmonized_dic = harmonized_variables.DataFrame()

In [ ]:
# Retrieve the dependent variable - total cholesterol
cholesterol_path = my_harmonized_dic.filter(like = 'Blood mass concentration of total cholesterol', axis = 0)
cholesterol_path = list(cholesterol_path.index)[0]
cholesterol_path

In [ ]:
# Create full list of concept paths with cholesterol_path removed
selected_vars = list(variablesDict['name'])
selected_vars.remove(cholesterol_path)

We are ready to create our query and retrieve the dataframe. This query will consist of two parts:
1. **Any record of `cholesterol_path`.** By performing an "any record of" filter on the `cholesterol_path`, we will filter out all participants that do not have total blood cholesterol measurements. This allows us to perform more meaningful statistical analysis on the data.
2. **Select all remaining harmonized variables.** We will then add all of the remaining harmonized variables to the query, which will allow us to retrieve this information.

In [ ]:
# Initialize a query
query = resource.query()

In [ ]:
query.anyof().add(cholesterol_path) # Use anyof for the cholesterol variable to filter out the NA values
query.select().add(selected_vars)
facts = query.getResultsDataFrame(low_memory=False)

In [ ]:
facts.head()

### 3. Data-management
Now that we have retrieved the data, we shall perform some data management steps to prepare for the statistical analysis. First, we will identify which variables are categorical and which are continuous using the "categorical" column of the `facts` dataframe. This is an example of how the PIC-SURE API greatly simplifies this step for the user, as categorizing variables can be tricky.

In [ ]:
categorical_varnames = my_harmonized_dic[my_harmonized_dic.categorical == True]
categorical_varnames = list(categorical_varnames.index)

continuous_varnames = my_harmonized_dic[my_harmonized_dic.categorical == False]
continuous_varnames = list(continuous_varnames.index)

In [ ]:
# remove cholesterol_path from continuous_varnames
continuous_varnames.remove(cholesterol_path) 
# remove subgroup concept path from categorical_varnames
categorical_varnames.remove("\\DCC Harmonized data set\\01 - Demographics\\A distinct subgroup within a study  generally indicating subjects who share similar characteristics due to study design. Subjects may belong to only one subcohort.\\")

To perform this PheWAS, we will frame two participant cohorts in the context of the dependent variable of interest. In this example, we are interested in blood cholesterol. However, `Blood mass concentation of total cholesterol` is a continuous variable. We shall convert this variable into a binary variable with two groups, Normal/Low and High cholesterol levels, by applying a [threshold of 200mg/dL](https://www.mayoclinic.org/diseases-conditions/high-blood-cholesterol/diagnosis-treatment/drc-20350806). 

In [ ]:
conditions = [
    list(facts[cholesterol_path] <= 200),
    list(facts[cholesterol_path] > 200)
]
outputs = ['Normal/Low', 'High']

In [ ]:
res = np.select(conditions, outputs)
facts['categorical_cholesterol'] = pd.Series(res)

We will also specify the variable name for the covariate we are interested in, in this case Sex.

In [ ]:
sex_path = list(facts.filter(regex = 'sex'))[0]
sex_path

We will also select our cohorts of interest. In this example, we are interested in participants from the Framingham Heart Study (FHS) and the Atherosclerosis Risk In Communities (ARIC) cohort. We can utilize the `A distinct subgroup within a study  generally indicating subjects who share similar characteristics due to study design. Subjects may belong to only one subcohort.` concept path in the DCC Harmonized data set to select the participants of interest.

In [ ]:
categorical_varnames = my_harmonized_dic[my_harmonized_dic.categorical == True]
categorical_varnames = list(categorical_varnames.index)

continuous_varnames = my_harmonized_dic[my_harmonized_dic.categorical == False]
continuous_varnames = list(continuous_varnames.index)

In [ ]:
facts['FHS_cohort'] = facts.iloc[:,1].str.contains('FHS')
facts['ARIC_cohort'] = facts.iloc[:,1].str.contains('ARIC')

fhs_subset = facts[facts.FHS_cohort == True]
aric_subset = facts[facts.ARIC_cohort == True]

### 4. Univariate statistical tests

From this point, each variable present in the `facts_dummies` dataset will be tested again the selected dependent variable, (ie presence or absence of COPD). 

Two different association tests will be carried out according to variables data types: 
- Mann-Whitney U test for continuous ones
- Fisher exact test for categorical ones

### LLR result

In [ ]:
from statsmodels.discrete.discrete_model import Logit

In [ ]:
independent_names = variablesDict["name"].tolist()
independent_names.remove(dependent_var_name)
dependent_var = facts[dependent_var_name].astype("category").cat.codes
dic_pvalues = {}
simple_index_variablesDict = variablesDict.set_index("name", drop=True)

In [ ]:
from scipy.linalg import LinAlgError
from statsmodels.tools.sm_exceptions import PerfectSeparationError
from tqdm import tqdm

In [ ]:
for independent_name in tqdm(independent_names, position=0, leave=True):
    matrix = facts.loc[:, [dependent_var_name, independent_name]]\
                  .dropna(how="any")
    if matrix.shape[0] == 0:
        dic_pvalues[independent_name] = np.NaN
        continue
    if simple_index_variablesDict.loc[independent_name, "categorical"]:
        matrix = pd.get_dummies(matrix,
                                columns=[independent_name],
                                drop_first=False)\
                    .iloc[:, 0:-1]
    dependent_var = matrix[dependent_var_name].cat.codes
    independent_var = matrix.drop(dependent_var_name, axis=1)\
                            .assign(intercept = 1)
    model = Logit(dependent_var, independent_var)
    try:
        results = model.fit(disp=0)
        dic_pvalues[independent_name] = results.llr_pvalue
    except (LinAlgError, PerfectSeparationError) as e:
        dic_pvalues[independent_name] = np.NaN

#### p-values distribution (univariate tests)

In [ ]:
pd.Series([v for v in dic_pvalues.values()]).plot.hist(bins=30)
plt.suptitle("Distribution of individual p-values",
             weight="bold",
            fontsize=15)

### 5. Multiple hypotheses testing correction: Bonferroni Method

In order to handle the multiple testing problem (increase in the probability of getting statistically significant associations), we will use the Bonferroni correction method. Although many other multiple-comparison corrections exist, Bonferroni is the most straightforward to use, because it doesn't require assumptions about variables correlation. Other PheWAS analysis also use False Discovery Rate controlling procedures ([see](https://en.wikipedia.org/wiki/False_discovery_rate)).

In a nutshell, Bonferonni allows to calculate a corrected "statistical significant threshold" according to the number of test performed. Every p-value below this threshold will be deemed statistically significant.

In [ ]:
%%capture
# Merging pvalues from different tests
df_pvalues = pd.DataFrame.from_dict(dic_pvalues, orient="index", columns=["pvalues"])\
.rename_axis("name")\
.reset_index(drop=False)

# Adding pvalues results as a new column to variablesDict
variablesDict = joining_variablesDict_onCol(variablesDict,
                                              df_pvalues,
                                              left_col="name",
                                              right_col="name")

adjusted_alpha = 0.05/len(variablesDict["pvalues"])
variablesDict["p_adj"] = variablesDict["pvalues"] / len(variablesDict["pvalues"])
variablesDict['log_p'] = -np.log10(variablesDict['pvalues'])
variablesDict = variablesDict.sort_index()
variablesDict["group"] = variablesDict.reset_index(level=2)["level_2"].values

In [ ]:
print("Bonferonni adjusted significance threshold: {0:.2E}".format(adjusted_alpha))

## 6. Result visualisations: Manhattan plot

Manhattan plot is the classical way to plot the results of a PheWAS analysis. It plots every tested phenotypical variables on the X-axis, against their *-log(pvalue)* on the Y-axis. The horizontal line represents the adjusted significance level threshold.

In [ ]:
mask = variablesDict["pvalues"].isna()
df_results = variablesDict.loc[~mask,:].copy().replace([np.inf, -np.inf], np.nan)
df_results = df_results.loc[~df_results["log_p"].isna().values,:]

#### Specific adjustment to make this specific plot looks nicer
####### to adapt when changing data or dependent variable
df_results = df_results.replace({"TLC": "Spirometry",
                                 "New Gold Classification": "Quantitative Analysis", 
                  "Other": "Demographics",
                                "Eligibility Form": "Sociodemography and Administration"})
group_order={'6MinWalk': 0,
 'CT Acquisition Parameters': 1,
 'CT Assessment Scoresheet': 2,
 'Demographics and Physical Characteristics': 3,
 'Longitudinal Analysis': 5,
 'Medical History': 4,
 'Medication History': 13,
 'Quantitative Analysis': 9,
 'Respiratory Disease': 6,
 'SF-36 Health Survey': 11,
 'Sociodemography and Administration': 12,
 'Spirometry': 7,
 'VIDA': 15}
df_results["group_order"] = df_results["group"].replace(group_order)
df_results = df_results.sort_values("group_order", ascending=True)
df_results["simplified_name"] = df_results["simplified_name"].str.replace("[0-9]+[A-z]*", "").to_frame()
###


fig = plt.figure()
ax = fig.add_subplot(111)
colors = plt.get_cmap('Set1')
x_labels = []
x_labels_pos = []

y_lims = (0, df_results["log_p"].max(skipna=True) + 50)
threshold_top_values = df_results["log_p"].sort_values(ascending=False)[0:6][-1]

df_results["ind"] = np.arange(1, len(df_results)+1)
df_grouped = df_results.groupby(('group'))
for num, (name, group) in enumerate(df_grouped):
    group.plot(kind='scatter', x='ind', y='log_p',color=colors.colors[num % len(colors.colors)], ax=ax, s=20)
    x_labels.append(name)
    x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2)) # Set label in the middle
    for n, row in group.iterrows():
        if row["log_p"] > threshold_top_values:
            ax.text(row['ind'] + 3, row["log_p"] + 0.05, row["simplified_name"], rotation=0, alpha=1, size=8, color="black")
                
ax.set_xticks(x_labels_pos)
ax.set_xticklabels(x_labels)
ax.set_xlim([0, len(df_results) +1])
ax.set_ylim(y_lims)
ax.set_ylabel('-log(p-values)', style="italic")
ax.set_xlabel('Phenotypes', fontsize=15)
ax.axhline(y=-np.log10(adjusted_alpha), linestyle=":", color="black", label="Adjusted Threshold")
plt.xticks(fontsize = 9,rotation=90)
plt.yticks(fontsize = 8)
plt.title("Statistical Association Between COPD Status and Phenotypes", 
          loc="center",
          style="oblique", 
          fontsize = 20,
         y=1)
xticks = ax.xaxis.get_major_ticks()
xticks[0].set_visible(False)
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles = handles, labels = labels, loc = "upper left")
plt.show()

Overall, it appears that most of the tested phenotypes covariates are above the adjusted threshold of significant association. However, it is not that surprising given the nature of our dependent variable: a lot of the collected phenotypic variables are correlated to the COPD status.

This code can be used directly with any other variable present in the variable Dictionary. Only the `dependent_var_name` variable need to be changed.